In [17]:
%pylab inline
import lasagne

Populating the interactive namespace from numpy and matplotlib


In [2]:
from fuel.transformers import ScaleAndShift, Cast, Flatten, Mapping
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme, ShuffledScheme

In [57]:
import numpy as np
import theano
import theano.tensor as T

In [56]:
import time

In [22]:
from fuel.datasets.cifar10 import CIFAR10

In [23]:
OFFSET = 10000

In [24]:
CIFAR10.default_transformers = (
    (ScaleAndShift, [2.0 / 255.0, -1], {'which_sources': 'features'}),
    (Cast, [np.float32], {'which_sources': 'features'}), 
    (Flatten, [], {'which_sources': 'features'}),
    (Mapping, [lambda batch: (b.T for b in batch)], {}) )

mnist_train = CIFAR10(("train",), subset=slice(None,OFFSET))
#this stream will shuffle the MNIST set and return us batches of 100 examples
mnist_train_stream = DataStream.default_stream(
    mnist_train,
    iteration_scheme=ShuffledScheme(mnist_train.num_examples, 100))
                                               

In [25]:
print "The streams return batches containing %s" % (mnist_train_stream.sources,)

print "Each training batch consits of a tuple containing:"
for element in next(mnist_train_stream.get_epoch_iterator()):
    print " - an array of size %s containing %s" % (element.shape, element.dtype)

The streams return batches containing (u'features', u'targets')
Each training batch consits of a tuple containing:
 - an array of size (3072, 100) containing float32
 - an array of size (1, 100) containing uint8


In [26]:
mnist_validation = CIFAR10(("train",), subset=slice(OFFSET, None))

# We will use larger portions for testing and validation
# as these dont do a backward pass and reauire less RAM.
mnist_validation_stream = DataStream.default_stream(
    mnist_validation, iteration_scheme=SequentialScheme(mnist_validation.num_examples, 250))
mnist_test = CIFAR10(("test",))
mnist_test_stream = DataStream.default_stream(
    mnist_test, iteration_scheme=SequentialScheme(mnist_test.num_examples, 250))

In [27]:
print "Validation/test batches consits of tuples containing:"
for element in next(mnist_test_stream.get_epoch_iterator()):
    print " - an array of size %s containing %s" % (element.shape, element.dtype)

Validation/test batches consits of tuples containing:
 - an array of size (3072, 250) containing float32
 - an array of size (1, 250) containing uint8


In [60]:
input_var = T.matrix('inputs')
target_var = T.ivector('targets')

In [62]:
l_in = lasagne.layers.InputLayer(shape=(None, 3072),
                                     input_var=input_var)

# Apply 20% dropout to the input data:
l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.2)

# Add a fully-connected layer of 800 units, using the linear rectifier, and
# initializing weights with Glorot's scheme (which is the default anyway):
l_hid1 = lasagne.layers.DenseLayer(
            l_in_drop, num_units=800,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())

# We'll now add dropout of 50%:
l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=0.5)

# Another 800-unit layer:
l_hid2 = lasagne.layers.DenseLayer(
            l_hid1_drop, num_units=800,
            nonlinearity=lasagne.nonlinearities.rectify)

# 50% dropout again:
l_hid2_drop = lasagne.layers.DropoutLayer(l_hid2, p=0.5)

# Finally, we'll add the fully-connected output layer, of 10 softmax units:
l_out = lasagne.layers.DenseLayer(
            l_hid2_drop, num_units=10,
            nonlinearity=lasagne.nonlinearities.softmax)


In [63]:
network = l_out

In [64]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

In [65]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.01, momentum=0.9)

In [66]:
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
test_loss = test_loss.mean()
# As a bonus, also create an expression for the classification accuracy:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

In [67]:
train_fn = theano.function([input_var, target_var], loss, updates=updates)

# Compile a second function computing the validation loss and accuracy:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [74]:
num_epochs = 100
train_stream = mnist_train_stream
validation_stream = mnist_validation_stream

In [75]:
for epoch in range(num_epochs):
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for inputs, targets in train_stream.get_epoch_iterator():
        train_err += train_fn(inputs.T, targets.ravel())
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in validation_stream.get_epoch_iterator():
        inputs, targets = batch
        err, acc = val_fn(inputs.T, targets.ravel())
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

Epoch 1 of 100 took 9.651s
  training loss:		1.774503
  validation loss:		1.680264
  validation accuracy:		41.57 %
Epoch 2 of 100 took 9.580s
  training loss:		1.714181
  validation loss:		1.670209
  validation accuracy:		41.91 %
Epoch 3 of 100 took 9.659s
  training loss:		1.667159
  validation loss:		1.649553
  validation accuracy:		42.79 %
Epoch 4 of 100 took 9.305s
  training loss:		1.636520
  validation loss:		1.638263
  validation accuracy:		43.01 %
Epoch 5 of 100 took 9.279s
  training loss:		1.596853
  validation loss:		1.620578
  validation accuracy:		43.23 %
Epoch 6 of 100 took 9.379s
  training loss:		1.569598
  validation loss:		1.595980
  validation accuracy:		44.06 %
Epoch 7 of 100 took 9.879s
  training loss:		1.544576
  validation loss:		1.590123
  validation accuracy:		44.39 %
Epoch 8 of 100 took 10.167s
  training loss:		1.517861
  validation loss:		1.565367
  validation accuracy:		45.07 %
Epoch 9 of 100 took 9.639s
  training loss:		1.491641
  validation loss:		1.575

KeyboardInterrupt: 